In [14]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

#!pip install autogluon
# remember to choose a python-version less than 3.11.0

In [15]:
x_train = pd.read_csv('cleaned_and_combined_data/x_train_combined.csv')
y_train = pd.read_csv('cleaned_and_combined_data/y_train_combined.csv')
x_test = pd.read_csv('cleaned_and_combined_data/x_test_combined.csv')

x_train_merged = pd.concat([x_train, y_train['pv_measurement']], axis=1)

In [16]:
# Convert the three columns into one column named 'location'
x_train_merged['location'] = x_train.apply(lambda row: 'A' if row['location_A'] == 1 else ('B' if row['location_B'] == 1 else 'C'), axis=1)
x_test['location'] = y_train.apply(lambda row: 'A' if row['location_A'] == 1 else ('B' if row['location_B'] == 1 else 'C'), axis=1)

# Drop the original three columns
x_train_merged.drop(['location_A', 'location_B', 'location_C'], axis=1, inplace=True)
x_test.drop(['location_A', 'location_B', 'location_C'], axis=1, inplace=True)

x_train_merged.head()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,day_of_year,day_of_year_sin,day_of_year_cos,direct_rad_3h_roll_avg,diffuse_rad_3h_roll_avg,direct_rad_6h_roll_avg,diffuse_rad_6h_roll_avg,direct_rad_x_sun_elevation,pv_measurement,location
0,2019-01-01 00:00:00,5.525,1.23975,1200.6750,0.0,0.0,1200.6750,0.0,275.150,0.0,...,1,0.017166,0.999853,0.0,0.0,0.0,0.0,-0.0,0.0,B
1,2019-01-01 01:00:00,5.425,1.23975,1131.4249,0.0,0.0,1131.4249,0.0,274.825,0.0,...,1,0.017166,0.999853,0.0,0.0,0.0,0.0,-0.0,0.0,B
2,2019-01-01 02:00:00,5.400,1.23850,1061.0000,0.0,0.0,1061.0000,0.0,274.800,0.0,...,1,0.017166,0.999853,0.0,0.0,0.0,0.0,-0.0,0.0,B
3,2019-01-01 03:00:00,5.350,1.23975,1021.1500,0.0,0.0,1021.1500,0.0,274.675,0.0,...,1,0.017166,0.999853,0.0,0.0,0.0,0.0,-0.0,0.0,B
4,2019-01-01 04:00:00,5.675,1.23750,1033.7000,0.0,0.0,1033.7000,0.0,275.500,0.0,...,1,0.017166,0.999853,0.0,0.0,0.0,0.0,-0.0,0.0,B


In [17]:
x_test['observed'] = x_test['calc_year'].isna().astype(int)
x_train_merged['observed'] = x_train_merged['calc_year'].isna().astype(int)

x_test.drop(['calc_year', 'calc_month', 'calc_day', 'calc_hour'], axis=1, inplace=True)
x_train_merged.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour'], axis=1, inplace=True)

x_train_merged.head()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,day_of_year_sin,day_of_year_cos,direct_rad_3h_roll_avg,diffuse_rad_3h_roll_avg,direct_rad_6h_roll_avg,diffuse_rad_6h_roll_avg,direct_rad_x_sun_elevation,pv_measurement,location,observed
0,2019-01-01 00:00:00,5.525,1.23975,1200.6750,0.0,0.0,1200.6750,0.0,275.150,0.0,...,0.017166,0.999853,0.0,0.0,0.0,0.0,-0.0,0.0,B,1
1,2019-01-01 01:00:00,5.425,1.23975,1131.4249,0.0,0.0,1131.4249,0.0,274.825,0.0,...,0.017166,0.999853,0.0,0.0,0.0,0.0,-0.0,0.0,B,1
2,2019-01-01 02:00:00,5.400,1.23850,1061.0000,0.0,0.0,1061.0000,0.0,274.800,0.0,...,0.017166,0.999853,0.0,0.0,0.0,0.0,-0.0,0.0,B,1
3,2019-01-01 03:00:00,5.350,1.23975,1021.1500,0.0,0.0,1021.1500,0.0,274.675,0.0,...,0.017166,0.999853,0.0,0.0,0.0,0.0,-0.0,0.0,B,1
4,2019-01-01 04:00:00,5.675,1.23750,1033.7000,0.0,0.0,1033.7000,0.0,275.500,0.0,...,0.017166,0.999853,0.0,0.0,0.0,0.0,-0.0,0.0,B,1


In [18]:
x_train_merged = x_train_merged.dropna(subset=['diffuse_rad:W'])
x_train_merged.drop(columns = ['ceiling_height_agl:m','cloud_base_agl:m','snow_density:kgm3'], inplace = True)
train_data = TimeSeriesDataFrame.from_data_frame(
    x_train_merged,
    id_column="location",
    timestamp_column="date_forecast"
)
missing_values = train_data.isnull().sum()
print(missing_values)


absolute_humidity_2m:gm3          0
air_density_2m:kgm3               0
clear_sky_energy_1h:J             0
clear_sky_rad:W                   0
dew_or_rime:idx                   0
dew_point_2m:K                    0
diffuse_rad:W                     0
diffuse_rad_1h:J                  0
direct_rad:W                      0
direct_rad_1h:J                   0
effective_cloud_cover:p           0
elevation:m                       0
fresh_snow_12h:cm                 0
fresh_snow_1h:cm                  0
fresh_snow_24h:cm                 0
fresh_snow_3h:cm                  0
fresh_snow_6h:cm                  0
is_day:idx                        0
is_in_shadow:idx                  0
msl_pressure:hPa                  0
precip_5min:mm                    0
precip_type_5min:idx              0
pressure_100m:hPa                 0
pressure_50m:hPa                  0
prob_rime:p                       0
rain_water:kgm2                   0
relative_humidity_1000hPa:p       0
sfc_pressure:hPa            

In [20]:
!pip install pyqlib -U


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached pyqlib-0.7.2.99-cp39-cp39-win_amd64.whl (454 kB)
  Using cached sacred-0.8.4-py2.py3-none-any.whl (107 kB)
  Obtaining dependency information for python-socketio from https://files.pythonhosted.org/packages/f8/bf/4790ed063ca2daa58fb20285fc3707218cf01e174209355d081d83094f6d/python_socketio-5.10.0-py3-none-any.whl.metadata
  Using cached python_socketio-5.10.0-py3-none-any.whl.metadata (3.2 kB)
  Obtaining dependency information for redis>=3.0.1 from https://files.pythonhosted.org/packages/0b/34/a01250ac1fc9bf9161e07956d2d580413106ce02d5591470130a25c599e3/redis-5.0.1-py3-none-any.whl.metadata
  Using cached redis-5.0.1-py3-none-any.whl.metadata (8.9 kB)
  Using cached python_redis_lock-4.0.0-py3-none-any.whl (12 kB)
  Obtaining dependency information for schedule>=0.6.0 from https://files.pythonhosted.org/packages/f0/1a/e5a65c08d708ee4c002f3f938ba69b2282af478755700c40f037eacc92ef/schedule-1.2.1-py2.py3-none-any.whl.metadata
  Using cached schedule-1.2.1-py2.py3-none-any.w

In [21]:
predictor = TimeSeriesPredictor(
    prediction_length=24*30,
    path="autogluon-ts",
    target="pv_measurement",
    eval_metric="RMSE",
    freq = 'H'
)

predictor.fit(
    train_data,
    presets="best_quality",
    time_limit=14400
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ ❱  1 predictor = TimeSeriesPredictor(                                                            │
│    2 │   prediction_length=24*30,                                                                │
│    3 │   path="autogluon-ts",                                                                    │
│    4 │   target="pv_measurement",                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: __init__() got an unexpected keyword argument 'freq'